In [1]:
import pandas as pd
import numpy as np

In [263]:
class NetPremium:
    
    def __init__(self, rate, i, **kwargs):
        self.v = 1/(1+i)
        self.l0 = 100000000
        
        self.gender = kwargs['성별']
        self.x = kwargs['연령']
        self.n = kwargs['보험기간']
        self.m = kwargs['납입기간']
        self.mpp = kwargs['납입주기']
        self.type = kwargs['종구분']
        self.rate = rate
        
    def __get_mortality_rate(self, rate, 상해질병구분, 담보명, 성별, x, k, options=None):
        """조건에 맞는 위험률 리턴"""
        
        if 상해질병구분 == 'accident':
            상해급수 = options['상해급수']
            return rate['accident'] \
                .query('담보명 == @담보명') \
                .query('상해급수 == @상해급수') \
                .query('성별 == @성별') \
                .drop(['담보명', '성별'], axis=1) \
                .sort_values(by='연령') \
                .set_index('연령') \
                ['위험률'] \
                .loc[x:x+k]
        elif 상해질병구분 == 'disease':
            return rate['disease'] \
                .query('담보명 == @담보명') \
                .query('성별 == @성별') \
                .drop(['담보명', '성별'], axis=1) \
                .sort_values(by='연령') \
                .set_index('연령') \
                ['위험률'] \
                .loc[x:x+k]
        else:
            raise Exception('상해질병구분 입력오류')
        
    def get_product(self):
        """상품명 리턴"""
        
        return '참좋은훼밀리플러스종합보험1901'
        
    def get_coverage(self):
        """담보명 리턴"""
        
        return '질병사망(301011)'
    
    def __calc_maintainer(self, x, n):
        """유지자(순보험료 분자 부분) 계산"""
        
        # 위험률
        q_질병사망률 = self.__get_mortality_rate(rate, 'disease', '질병사망률', self.gender, self.x, self.n)
        
        # lx, Cx, Mx
        lx = pd.Series(data=0, index=np.arange(x, x+n+1))
        lx.index.name = '연령'
        lx[x] = self.l0
        for t in range(n):
            lx[x+t+1] = lx[x+t]*(1-q_질병사망률[x+t])
        Cx = lx*q_질병사망률*v**np.arange(x, x+n+1)
        Mx = Cx[::-1].cumsum()[::-1]
        
        return Mx[x]-Mx[x+n]
    
    def __calc_payer(self, x, m, mpp, **kwargs):
        """납입자(순보험료 분모 부분) 계산"""
        
        
        if kwargs['종구분'] == '1종':
            # 위험률
            q_질병사망률 = self.__get_mortality_rate(rate, 'disease', '질병사망률', self.gender, self.x, self.m)
            q_상해1급일반상해후유장해발생률_80이상 = self.__get_mortality_rate(rate, 'accident', '일반상해후유장해발생률(80%이상)', self.gender, self.x, self.m,options={'상해급수': '1급'})
            q_질병후유장해발생률_80이상 = self.__get_mortality_rate(rate, 'disease', '질병후유장해발생률(80%이상)', self.gender, self.x, self.m)
            q_기타피부암및갑상선암이외의암발생률 = self.__get_mortality_rate(rate, 'disease', '기타피부암및갑상선암이외의암발생률', self.gender, self.x, self.m)
            q_뇌졸중발생률 = self.__get_mortality_rate(rate, 'disease', '뇌졸중발생률', self.gender, self.x, self.m)
            q_급성심근경색증발생률 = self.__get_mortality_rate(rate, 'disease', '급성심근경색증발생률', self.gender, self.x, self.m)
            q_말기신부전증발생률 = self.__get_mortality_rate(rate, 'disease', '말기신부전증발생률', self.gender, self.x, self.m)
            q_말기폐질환발생률 = self.__get_mortality_rate(rate, 'disease', '말기폐질환발생률', self.gender, self.x, self.m)
            q_말기간경화증발생률 = self.__get_mortality_rate(rate, 'disease', '말기간경화증발생률', self.gender, self.x, self.m)
            self.q_j = [q_상해1급일반상해후유장해발생률_80이상, q_질병후유장해발생률_80이상, q_기타피부암및갑상선암이외의암발생률, q_뇌졸중발생률, q_급성심근경색증발생률, q_말기신부전증발생률, q_말기폐질환발생률, q_말기간경화증발생률]

            # lpx, Dpx, Npx
            lpx = pd.Series(data=0, index=np.arange(x, x+m+1))
            lpx.index.name = '연령'
            lpx[x] = self.l0
            for t in range(m):
                lpx[x+t+1] = lpx[x+t]*(1-q_질병사망률[x+t])
                for j in range(8):
                    Dj = 3/4 if t==0 and j==2 else 1
                    lpx[x+t+1] *= (1-Dj*q_j[j][x+t]*(1-0.5*q_질병사망률[x+t])/(1-q_질병사망률[x+t]))
            Dpx = lpx*v**np.arange(x, x+m+1)
            Npx = Dpx[::-1].cumsum()[::-1]
            NN = mpp*((Npx[x]-Npx[x+m])-(mpp-1)/(2*mpp)*(Dpx[x]-Dpx[x+m]))

            return NN
        else:
            raise Exception('종구분 입력오류')
    
    def calc_net_premium(self):
        """순보험료 계산"""
        
        return self.__calc_maintainer(self.x, self.n)/self.__calc_payer(self.x, self.m, self.mpp, 종구분=self.type)

In [266]:
# 테스트
x, n, m, mpp = 30, 20, 10, 12
100000000*NetPremium(rate, 0.02, 성별='남자', 연령=x, 보험기간=n, 납입기간=m, 납입주기=mpp, 종구분='1종').calc_net_premium()

12049.248899247605

In [202]:
# 데이터 불러오기
disease = pd.read_excel('data/질병사망(301011)_기초데이터_20200806.xlsx', sheet_name='질병', dtype={'연령': int, '위험률': float, '성별': str})
accident = pd.read_excel('data/질병사망(301011)_기초데이터_20200806.xlsx', sheet_name='상해', dtype={'연령': int, '위험률': float, '성별': str, '상해급수': str})
rate = {'disease': disease, 'accident': accident}